In [5]:
import os
from pathlib import Path
from boutdata import collect
from matplotlib import pyplot as plt
import numpy as np
from netCDF4 import Dataset
import glob
from scipy.ndimage import zoom

In [ ]:
# IMPORTANT: Running this notebok requires files of a size that are not appropiate for a GitHub repository. You need to install the probe data locally and keep it as an untracked folder.

s1 = r"/Volumes/TOSHIBA EXT/Deep Learning/Data/Probe"
s2 = r"/Volumes/TOSHIBA EXT/Deep Learning/Data/Simulation"
e = r"data_numpy"

print(Path("/Volumes/TOSHIBA EXT/Deep Learning/Data/Probe").exists())


DATA_LOCATION_PROBE = Path(s1)
DATA_LOCATION_SIMULATION = Path(s2)
EXPORT_LOCATION = Path(e)

field_list = ['n'] # Extract density
probe_list = ['0', '1', '2', '3', '4', '5', '6'] # We need all 7 probes


True


In [7]:
probe_data = {}
for p in probe_list:
    probe_data[p] = {}

    probe_vars = [f"{f}{p}" for f in field_list]
    probe_vars.append('t_array')
    for var in probe_vars:
        for file in glob.glob(f'{DATA_LOCATION_PROBE}/BOUT.fast.*'):
            try:
                with Dataset(file) as f:
                    probe_data[p][var] = f[var][:]
                    print(f"data {var} loaded from {file.split('/')[-1]}")
                    break
            except:
                continue

print(probe_data)

data n0 loaded from BOUT.fast.12.nc
data t_array loaded from BOUT.fast.0.nc
data t_array loaded from BOUT.fast.0.nc
data n1 loaded from BOUT.fast.19.nc
data n1 loaded from BOUT.fast.19.nc
data t_array loaded from BOUT.fast.0.nc
data t_array loaded from BOUT.fast.0.nc
data n2 loaded from BOUT.fast.25.nc
data n2 loaded from BOUT.fast.25.nc
data t_array loaded from BOUT.fast.0.nc
data t_array loaded from BOUT.fast.0.nc
data n3 loaded from BOUT.fast.32.nc
data n3 loaded from BOUT.fast.32.nc
data t_array loaded from BOUT.fast.0.nc
data t_array loaded from BOUT.fast.0.nc
data n4 loaded from BOUT.fast.38.nc
data n4 loaded from BOUT.fast.38.nc
data t_array loaded from BOUT.fast.0.nc
data t_array loaded from BOUT.fast.0.nc
data n5 loaded from BOUT.fast.44.nc
data n5 loaded from BOUT.fast.44.nc
data t_array loaded from BOUT.fast.0.nc
data t_array loaded from BOUT.fast.0.nc
data n6 loaded from BOUT.fast.51.nc
data n6 loaded from BOUT.fast.51.nc
data t_array loaded from BOUT.fast.0.nc
{'0': {'n0':

In [8]:
nx, nz = collect('nx', path=DATA_LOCATION_SIMULATION), collect('nz', path=DATA_LOCATION_SIMULATION)

probe_pos = {}
for p in probe_list:
    probe_pos[p] = {}
    frac = 0.1*(int(p)+2)
    probe_pos[p]['x'] = round(frac*int(nx)/4)
    probe_pos[p]['z'] = round(0.5*int(nz)/4)

print(probe_pos)

{'0': {'x': 26, 'z': 64}, '1': {'x': 39, 'z': 64}, '2': {'x': 51, 'z': 64}, '3': {'x': 64, 'z': 64}, '4': {'x': 77, 'z': 64}, '5': {'x': 90, 'z': 64}, '6': {'x': 103, 'z': 64}}


In [ ]:
sim_data = {}
sim_data['t_array'] = collect('t_array', path = DATA_LOCATION_SIMULATION)
for f in field_list:
    sim_data[f] = collect(f, path = DATA_LOCATION_SIMULATION)

mxsub = 8 mysub = 1 mz = 512

nxpe = 64, nype = 1, npes = 64

mxsub = 8 mysub = 1 mz = 512

nxpe = 64, nype = 1, npes = 64

Reading from 0: [0-8][0-0] -> [0-8][0-0]

Reading from 0: [0-8][0-0] -> [0-8][0-0]

Reading from 1: [1-8][0-0] -> [9-16][0-0]

Reading from 1: [1-8][0-0] -> [9-16][0-0]

Reading from 2: [1-8][0-0] -> [17-24][0-0]

Reading from 2: [1-8][0-0] -> [17-24][0-0]

Reading from 3: [1-8][0-0] -> [25-32][0-0]

Reading from 3: [1-8][0-0] -> [25-32][0-0]

Reading from 4: [1-8][0-0] -> [33-40][0-0]

Reading from 4: [1-8][0-0] -> [33-40][0-0]



In [ ]:
zoom_t_start = 30000 
zoom_t_end = 35000
zoom_y_min = 0.0
zoom_y_max = 0.9

print(len(sim_data['t_array']))
print_plots = True
if print_plots == True:
    for p in ['3']:
        
        for f in field_list:
            fig, ax = plt.subplots()
        
            ax.plot(probe_data[p]['t_array'], probe_data[p][f"{f}{p}"], label = 'probe data')
            ax.plot(sim_data['t_array'], sim_data[f][:, probe_pos[p]['x'], 0, probe_pos[p]['z']], "x", label = 'simulation data')
            ax.legend(loc="upper left")
        
            ax_inset = ax.inset_axes([1.1, 0.1, 0.8, 0.8])
        
            ax_inset.plot(probe_data[p]['t_array'], probe_data[p][f"{f}{p}"], label = 'probe data')
            ax_inset.plot(sim_data['t_array'], sim_data[f][:, probe_pos[p]['x'], 0, probe_pos[p]['z']], "x", label = 'simulation data')
        
            ax_inset.set_xlim(zoom_t_start, zoom_t_end)
            ax_inset.set_ylim(zoom_y_min, zoom_y_max)
            
            ax.indicate_inset_zoom(ax_inset, edgecolor="black")
        
            fig.suptitle(f"probe: {p}, field: {f}")

501


In [ ]:
# Create a dictionary to store probe data in a saveable format
probe_data_export = {}

for p in probe_list:

    position = probe_pos[p]  # a dictionary with x and z
    time_measurement = probe_data[p]['t_array']
    data_measurement = probe_data[p][f"{p}n"]
    data = {
        'probe_num': int(p),
        'position': position,
        't_array': time_measurement
    }
    EXPORT_LOCATION.mkdir(parents=True, exist_ok=True)
    export_path = EXPORT_LOCATION / f'probe{p}_data128.npy'
    np.save(export_path, data)

    print(f"  probe_{p}: position={data['position']}, measurement shape={data['measurement'].shape}")

# # Save as numpy .npy file
# EXPORT_LOCATION.mkdir(parents=True, exist_ok=True)
# export_path = EXPORT_LOCATION / 'probe_data128.npy'
# np.save(export_path, probe_data_export)

# # print(f"Saved probe data to {export_path}")
# # print(f"Data contains {len(probe_data_export)} probes")
# # for p in probe_list:
# #     # print(f"  probe_{p}: position={probe_data_export[p]['position']}, measurement shape={probe_data_export[p]['measurement'].shape}")

  probe_0: position={'x': 26, 'z': 64}, measurement shape=(7439539,)
  probe_1: position={'x': 39, 'z': 64}, measurement shape=(7439539,)
  probe_2: position={'x': 51, 'z': 64}, measurement shape=(7439539,)
  probe_3: position={'x': 64, 'z': 64}, measurement shape=(7439539,)
  probe_4: position={'x': 77, 'z': 64}, measurement shape=(7439539,)
  probe_2: position={'x': 51, 'z': 64}, measurement shape=(7439539,)
  probe_3: position={'x': 64, 'z': 64}, measurement shape=(7439539,)
  probe_4: position={'x': 77, 'z': 64}, measurement shape=(7439539,)
  probe_5: position={'x': 90, 'z': 64}, measurement shape=(7439539,)
  probe_6: position={'x': 103, 'z': 64}, measurement shape=(7439539,)
  probe_5: position={'x': 90, 'z': 64}, measurement shape=(7439539,)
  probe_6: position={'x': 103, 'z': 64}, measurement shape=(7439539,)
